# Pyxem template matching example

As usual, start with loading required dependencies

In [ ]:
# You might have tk installed instead of qt
%matplotlib qt
import math
import numpy as np

import matplotlib.pyplot as plt

import pyxem as pxm
from pyxem.libraries.structure_library import StructureLibrary
from pyxem.generators.indexation_generator import IndexationGenerator

import diffpy.structure

from transforms3d.axangles import axangle2mat
from transforms3d.euler import euler2mat
from transforms3d.euler import mat2euler

import warnings
# Silence some future warnings and user warnings (float64 -> uint8)
# in skimage when calling remove_background with h-dome (below)
# Should really be fixed elsewhere.
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

## Parameters and loading
Let's start defining some variables and parameters for later use. The data paths should be updated to match your system.

Local, absolute paths obviously has to be changed.

In [ ]:
# Data to be analyzed
data_dir = 'D:/Dokumenter/MTNANO/Prosjektoppgave/SPED_data_GaAs_NW/'
in_file = data_dir + 'gen/Julie_180510_SCN45_FIB_a_pyxem_sample.hdf5'

# Structure files defining the atomic structure. Loaded using diffpy, could be e.g. .cif files.
structure_zb_file = 'D:\\Dokumenter/MTNANO/Prosjektoppgave/Data/Gen/NN_test_data/GaAs_mp-2534_conventional_standard.cif'
structure_wz_file = 'D:\\Dokumenter/MTNANO/Prosjektoppgave/Data/Gen/NN_test_data/GaAs_mp-8883_conventional_standard.cif'

# Some experimental parameters
beam_energy_keV = 200
max_excitation_error = 1/80  # Ångström^{-1}, extent of relrods in reciprocal space. Inverse of specimen thickness is a starting point
nm_per_pixel = 1.12  # Real space step length
reciprocal_angstrom_per_pixel = 0.033  # Reciprocal calibration

direct_beam_mask_radius = 3  # pixels. Radius of direct beam, for masking
rotation_list_resolution = np.deg2rad(1)  # Radians. Resolution of generated pattern library

Actually load the files. First, the structure files defining the crystal structure is loaded.

Then load the SPED dataset. The file is lazy-loaded and then cut. This ensures that only required areas are loaded from disk to memory.

The data type is changed to float and some metadata is set. The call to `pxm.ElectronDiffraction` converts the lazy hyperspy signal to an actual pyxem object. The metadata from the file has to be copied manually. The constructor probably should have done so automatically, but it does not.

In [ ]:
structure_zb = diffpy.structure.loadStructure(structure_zb_file)
structure_wz = diffpy.structure.loadStructure(structure_wz_file)

dp = pxm.load(in_file, lazy=True)
# dp = dp.inav[95:100, 110:150]  # ZB1, ZB2 and transition
dp = dp.inav[95:100, 30:75]  # WZ, ZB1, ZB2
if dp.data.dtype != 'float64':
    dp.change_dtype('float64')

# Convert to a pyxem ElectronDiffraction, conserve the metadata and add some more
dp_metadata = dp.metadata
dp = pxm.ElectronDiffraction(dp)
dp.metadata = dp_metadata
dp.set_scan_calibration(nm_per_pixel)
dp.set_diffraction_calibration(reciprocal_angstrom_per_pixel)

## Data cleaning
After loading the signal, do some cleaning of the data.

I would like to start by rotating to a common direction. This simplifies the pattern matching, since one rotational degree of freedom is removed. For this dataset, the rotation is 41°. For this to work, the template library would have to be generated to the same base direction, but the current implementation does not. In this dataset, there is an extra difficulty in that ZB appears in two rotations. It should be a 120° in real space, but the difference is about 70° (110°) in the diffraction pattern. This might be because of stretching of the pattern? To be investigated.

Next, the pattern should be centered. The `center_direct_beam` assumes the center beam is the strongest, which is not always the case in this dataset. The beam is already centered enough to give matches, but you could compare the results with and without the call to `center_direct_beam`.

Pyxem offers multiple methods of removing background noise. Parameter testing was done in a separate notebook to keep this one clean. `h-dome` with a parameter of `h=0.55` works fine on this dataset.

Finally, the center beam could be masked. Since all diffraction patterns have this beam, there is no reason to match against it. The alternative, which is done here, is just generating patterns without the center beam. As long as the diffraction pattern is centered and the spots are not too close, it does not matter that there is an extra spot. The current implementation of the template matching only checks spots present in the generated template.

The data is also normalized to a maximum intensity.

In [ ]:
#dp.apply_affine_transformation(euler2mat(0, 0, np.deg2rad(41), 'sxyz'))
#dp.center_direct_beam(radius_start=2, radius_finish=5)
dp.apply_affine_transformation(np.array([
    [0.95, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
    ]))
dp = dp.remove_background('h-dome', h=0.55)
#mask = dp.get_direct_beam_mask(radius=direct_beam_mask_radius)
#dp.data *= np.invert(mask)
dp.data *= 1 / dp.data.max()

## Generate pattern library
Now, define the functions for generating the pattern library. See function docstrings for an explanation.

In [ ]:
def angle_between_directions(structure,
                            direction1,
                            direction2):
    """Returns the angle in radians between two crystal directions in the given structure.
    
    Arguments:
        structure: diffpy.structure.Structure whose lattice gives the coordinate system.
        direction1, direction2: Directions given with three components using the structure 
            lattice as a coordinate system.
    Returns:
        Angle between direction1 and direction2 in radians.
    """
    a = structure.lattice.a
    b = structure.lattice.b
    c = structure.lattice.c
    alpha = np.deg2rad(structure.lattice.alpha)
    beta = np.deg2rad(structure.lattice.beta)
    gamma = np.deg2rad(structure.lattice.gamma)
    
    u1 = direction1[0]
    v1 = direction1[1]
    w1 = direction1[2]
    
    u2 = direction2[0]
    v2 = direction2[1]
    w2 = direction2[2]
    
    L = a**2*u1*u2 + b**2*v1*v2 + c**2*w1*w2 \
        + b*c*(v1*w2 + w1*v2)*math.cos(alpha) \
        + a*c*(w1*u2 + u1*w2)*math.cos(beta) \
        + a*b*(u1*v2 + v1*u2)*math.cos(gamma)
    
    I1 = np.sqrt(a**2 * u1**2 + b**2*v1**2 + c**2*w1**2 \
        + 2*b*c*v1*w1*math.cos(alpha) \
        + 2*a*c*w1*u1*math.cos(beta) \
        + 2*a*b*u1*v1*math.cos(gamma))
    
    I2 = np.sqrt(a**2 * u2**2 + b**2*v2**2 + c**2*w2**2 \
        + 2*b*c*v2*w2*math.cos(alpha) \
        + 2*a*c*w2*u2*math.cos(beta) \
        + 2*a*b*u2*v2*math.cos(gamma))

    return math.acos(min(1, L/(I1*I2)))


def generate_complete_rotation_list(structure, corner_a, corner_b, corner_c, resolution, psi_angles):
    """Generate a rotation list covering the inverse pole figure specified by three
        corners in cartesian coordinates.
    
    Arguments:
        structure: diffpy.structure.Structure, used for calculating angles
        corner_a, corner_b, corner_c: The three corners of the inverse pole
            figure given by three coordinates in the coordinate system
            specified by the structure lattice.
        resolution: Angular resolution in radians of the generated rotation
            list.
        psi_angles: np.array with angles in radians for final rotation angle.
    
    Returns:
        Rotations covering the inverse pole figure given as a `np.array` of Euler
            angles in degress. This `np.array` can be passed directly to pyxem.
    """
    # Start defining some angles and normals from the given corners
    angle_a_to_b = angle_between_directions(structure, corner_a, corner_b)
    angle_a_to_c = angle_between_directions(structure, corner_a, corner_c)
    angle_b_to_c = angle_between_directions(structure, corner_b, corner_c)
    axis_a_to_b = np.cross(corner_a, corner_b)
    axis_a_to_c = np.cross(corner_a, corner_c)

    # Input validation. The corners have to define a non-degenerate triangle
    if np.count_nonzero(axis_a_to_b) == 0:
        raise ValueError('Directions a and b are parallel')
    if np.count_nonzero(axis_a_to_c) == 0:
        raise ValueError('Directions a and c are parallel')
    
    # Find the maxiumum number of points we can generate, given by the
    # resolution, then allocate storage for them. For the theta direction,
    # ensure that we keep the resolution also along the direction to the corner
    # b or c farthest away from a.
    psi_count = psi_angles.shape[0]
    theta_count = math.ceil(max(angle_a_to_b, angle_a_to_c) / resolution)
    phi_count = math.ceil(angle_b_to_c / resolution)
    rotations = np.zeros((theta_count, phi_count, psi_count, 3))

    # For each theta_count angle theta, evenly spaced
    for i, (theta_b, theta_c) in enumerate(zip(
        np.linspace(0, angle_a_to_b, theta_count),
        np.linspace(0, angle_a_to_c, theta_count))):
        # Define the corner local_b at a rotation theta from corner_a toward
        #   corner_b on the circle surface. Similarly, define the corner
        #   local_c at a rotation theta from corner_a toward corner_c

        rotation_a_to_b = axangle2mat(axis_a_to_b, theta_b)
        rotation_a_to_c = axangle2mat(axis_a_to_c, theta_c)
        local_b = np.dot(corner_a, rotation_a_to_b)
        local_c = np.dot(corner_a, rotation_a_to_c)

        # Then define an axis and a maximum rotation to create a great cicle
        # arc between local_b and local_c. Ensure that this is not a degenerate
        # case where local_b and local_c are coincident.
        angle_local_b_to_c = angle_between_directions(structure, local_b, local_c)
        axis_local_b_to_c = np.cross(local_b, local_c)
        if np.count_nonzero(axis_local_b_to_c) == 0:
            # Theta rotation ended at the same position. First position, might
            # be other cases?
            axis_local_b_to_c = corner_a
        axis_local_b_to_c /= np.linalg.norm(axis_local_b_to_c)

        # Generate points along the great circle arc with a distance defined by
        # resolution.
        phi_count_local = math.ceil(angle_local_b_to_c / resolution)
        for j, phi in enumerate(np.linspace(0, angle_local_b_to_c, phi_count_local)):
            rotation_phi = axangle2mat(axis_local_b_to_c, phi)
            for k, psi in enumerate(psi_angles):
                rotation_psi = axangle2mat((0, 0, 1), psi)
                
                # Combine the rotations. Order is important. The structure is
                # multiplied from the left in diffpy, and we want to rotate by
                # phi first.
                rotation = np.matmul(rotation_phi, np.matmul(rotation_a_to_b, rotation_psi))
                # Finally, convert to Euler angles in degrees for passing
                # to pyxem (which then immediately converts them back to
                # a matrix).
                rotations[i, j, k] = np.rad2deg(mat2euler(rotation, 'rzxz'))
    
    # Before returning, flatten to one-dimensional list of Euler angle triplets and
    # remove duplicate items.
    return np.unique(rotations.reshape(-1, 3), axis=0)


def uvtw_to_uvw(u, v, t, w):
    """Convert Miller-Bravais indices to Miller indices.
    
    Arguments:
        u, v, t, w: Four integers specifying a direction
            using Miller-Bravais indices.
    Returns:
        u, v, w: Three integers specifying the same direction
            using Miller indices, reduced by dividing by greatest
            common denominator.
    """
    U, V, W = 2*u + v, 2*v + u, w
    common_factor = math.gcd(math.gcd(U, V), W)
    return tuple((int(x/common_factor)) for x in (U, V, W))


def direction_to_cartesian(structure, direction):
    """Convert a vector from a coordinate system defined by structure.lattice
    to cartesian.
    
    Arguments:
        structure: diffpy.structure.Structure with a lattice defining the coordinate system
        direction: 3D vector to be converted
        
    Returns:
        3D vector : np.array. Direction with basis changed to cartesian.
    """
    a = structure.lattice.a
    b = structure.lattice.b
    c = structure.lattice.c
    alpha = np.deg2rad(structure.lattice.alpha)  # angle a to c
    beta  = np.deg2rad(structure.lattice.beta)   # angle b to c
    gamma = np.deg2rad(structure.lattice.gamma)  # angle a to b

    cos_alpha = math.cos(alpha)
    cos_beta  = math.cos(beta)
    cos_gamma = math.cos(gamma)
    sin_gamma = math.sin(gamma)

    factor_e3_0 = cos_beta
    factor_e3_1 = (cos_alpha - cos_beta*cos_gamma)/sin_gamma
    factor_e3_2 = math.sqrt(1 - np.dot(factor_e3_0, factor_e3_0) - np.dot(factor_e3_1, factor_e3_1))

    transform = np.array([
        [a, b*cos_gamma, c*factor_e3_0],
        [0, b*sin_gamma, c*factor_e3_1],
        [0, 0,           c*factor_e3_2]
    ])
    return np.dot(transform, direction)

Now use the functions defined above to generate a rotation list covering the inverse pole figure for the ZB and WZ structures. The ZB pole figure is defined between the corners $(0, 0, 1), (1, 0, 1), (1, 1, 1)$, and the WZ pole figure is defined between $(0, 0, 0, 1), (1, 1, -2, 0), (1, 0, -1, 0)$. Generate two sets of rotations for ZB, since the dataset contains twins. We could also generate two separate lists and treat them as separate phases.

A probably better version of this is to generate a relatively coarse rotation list, find the best `n` matches and then generate local rotation lists around these directions and phases.

In [ ]:
rotation_list_ZB = generate_complete_rotation_list(
    structure_zb,
    (0, 0, 1),
    (1, 0, 1),
    (1, 1, 1),
    rotation_list_resolution,
    np.deg2rad((84.6, 15)))
rotation_list_WZ = generate_complete_rotation_list(
    structure_wz,
    direction_to_cartesian(structure_wz, uvtw_to_uvw(0, 0, 0, 1)),
    direction_to_cartesian(structure_wz, uvtw_to_uvw(1, 1, -2, 0)),
    direction_to_cartesian(structure_wz, uvtw_to_uvw(1, 0, -1, 0)),
    rotation_list_resolution,
    np.deg2rad((109,)))

Create the structure library and diffraction generator using paramters specified at the top.

In [ ]:
phases = ['ZB', 'WZ']
structure_library = StructureLibrary(
        phases,
        [structure_zb, structure_wz],
        [rotation_list_ZB, rotation_list_WZ])
gen = pxm.DiffractionGenerator(beam_energy_keV, max_excitation_error=max_excitation_error)
library_generator = pxm.DiffractionLibraryGenerator(gen)
target_pattern_dimension_pixels = dp.axes_manager.signal_shape[0]
half_pattern_size = target_pattern_dimension_pixels // 2
reciprocal_radius = reciprocal_angstrom_per_pixel*(half_pattern_size - 1)

And finally generate the actual template library.

In [ ]:
diffraction_library = library_generator.get_diffraction_library(
    structure_library,
    calibration=reciprocal_angstrom_per_pixel,
    reciprocal_radius=reciprocal_radius,
    half_shape=(half_pattern_size, half_pattern_size),
    with_direct_beam=False)

## Correlation
Given the `diffraction_library` defined above, the `IndexationGenerator` finds the correlation between all patterns in the library and each experimental pattern, and returns the `n_largest` with highest correlation.

In [ ]:
indexer = IndexationGenerator(dp, diffraction_library)
indexation_results = indexer.correlate(n_largest=4, keys=phases)

Now, visualize the results.

In [ ]:
crystal_map = indexation_results.get_crystallographic_map()
crystal_map.get_phase_map().plot()
crystal_map.get_distance_from_modal_angle().plot()

Let's look at the best match. For now, single position at a time. https://github.com/pyxem/pyxem/pull/293 should fix the built-in solution. Instead, first get the matches and store them in peaks.

In [ ]:
peaks = []
for indexation_result in indexation_results:
    single_match_result = indexation_result.data
    best_fit = single_match_result[np.argmax(single_match_result[:, 4])]
    phase_name = phases[int(best_fit[0])]
    library_entry = diffraction_library.get_library_entry(phase=phase_name, angle=(best_fit[1], best_fit[2], best_fit[3]))
    #library_entry = diffraction_library.get_library_entry(phase='WZ', angle=(120.954560921187, 90.0, -41))  # To look at a specific phase and rotation (angles from rotation list)
    peaks.append((library_entry['pixel_coords'], library_entry['intensities'], [phase_name, *best_fit[1:4]]))
peaks = np.array(peaks).reshape(dp.data.shape[0], dp.data.shape[1], 3)

Then, plot the image and write the phase and angle

In [ ]:
dp.set_diffraction_calibration(reciprocal_angstrom_per_pixel)
x = 0
y = 30
plt.figure(50)
plt.cla()
plt.imshow(dp.inav[x, y])
plt.scatter(peaks[y, x, 0][:, 0], peaks[y, x, 0][:, 1], marker='x', c=peaks[y, x, 1], cmap='viridis')
print('Best fit:', peaks[y, x, 2])